# Building a Multi-Agent Deep Researcher

## Setup

### Install Packages

In [1]:
# !uv pip install crewai crewai-tools linkup-sdk

Resolve this issue in CrewAI before proceeding: https://github.com/crewAIInc/crewAI/issues/2606

### API Keys

We'll need the Linkup API key for building our Deep Researcher, you can sign up [here](https://app.linkup.so/sign-up).

In [25]:
import os, json, re, getpass
from dotenv import load_dotenv
from IPython.display import display, Markdown
load_dotenv(override=True)

True

In [3]:
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

In [4]:
#Check for OpenAI API Key
_set_env("OPENAI_API_KEY")

### Libraries

In [5]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
from typing import Type
from pydantic import BaseModel, Field
from linkup import LinkupClient
from crewai import Agent, Task, Crew, Process, LLM
from crewai.tools import BaseTool

## Selecting Components

### Chat Model

Check [this](https://docs.crewai.com/en/concepts/llms) for available LLM integrations.

In [8]:
llm = LLM(
    # model="groq/llama3-8b-8192",
    model="openai/gpt-4o-mini",
    # model ="groq/llama-3.2-90b-text-preview"
    # temperature=0.5,
    # max_completion_tokens=1024,
    # top_p=0.9,
    # stop=None,
    # stream=False,
)

ERROR! Session/line number was not unique in database. History logging moved to new session 1708


In [9]:
llm.call("hello")

'Hello! How can I assist you today?'

### Define Linkup Search Tool

In [10]:
class LinkupSearchInput(BaseModel):
    """Input schema for Linkup Search Tool."""
    query: str = Field(description="The search query to perform")
    depth: str = Field(default="standard",
                       description="Depth of search: 'standard' or 'deep'")
    output_type: str = Field(
        default="searchResults", description="Output type: 'searchResults', 'sourcedAnswer', or 'structured'")

In [11]:
class LinkupSearchTool(BaseTool):
    name: str = "Linkup Search"
    description: str = "Search the web for information using Linkup and return comprehensive results"
    args_schema: Type[BaseModel] = LinkupSearchInput

    def __init__(self):
        super().__init__()

    def _run(self, query: str, depth: str = "standard", output_type: str = "searchResults") -> str:
        """Execute Linkup search and return results."""
        try:
            # Initialize Linkup client with API key from environment variables
            linkup_client = LinkupClient(api_key=os.getenv("LINKUP_API_KEY"))

            # Perform search
            search_response = linkup_client.search(
                query=query,
                depth=depth,
                output_type=output_type
            )

            return str(search_response)
        except Exception as e:
            return f"Error occurred while searching: {str(e)}"

In [12]:
# Initialize tools
linkup_search_tool = LinkupSearchTool()

Generating description...
Method called by: LinkupSearchTool
{'query': {'description': 'The search query to perform', 'type': 'str', 'required': True}, 'depth': {'description': "Depth of search: 'standard' or 'deep'", 'type': 'str', 'required': True}, 'output_type': {'description': "Output type: 'searchResults', 'sourcedAnswer', or 'structured'", 'type': 'str', 'required': True}}
Tool Name: Linkup Search
Tool Arguments: {'query': {'description': 'The search query to perform', 'type': 'str', 'required': True}, 'depth': {'description': "Depth of search: 'standard' or 'deep'", 'type': 'str', 'required': True}, 'output_type': {'description': "Output type: 'searchResults', 'sourcedAnswer', or 'structured'", 'type': 'str', 'required': True}}
Tool Description: Search the web for information using Linkup and return comprehensive results
Usage Instructions: To use this tool (Linkup Search), you must provide the following parameters with their corresponding data types:
- query: The search query 

## Setting up Crew of Agents

In CrewAI, you define **agents** as independent workers, each with a specific job and objective. You then assign them **tasks**, which are detailed instructions on what they need to do to reach that objective. You can also assign **tools** directly as tasks for agents to use.

**Let's first create our CrewAI agents with role, main goal/objective, and backstory/personality.**

### Web Searcher Agent

In [13]:
web_searcher = Agent(
        role="Web Searcher",
        goal="Find the most relevant information on the web, along with source links (urls) for this user query: {query}.",
        backstory="An expert at formulating search queries and retrieving relevant information. Passes the results to the 'Research Analyst' only.",
        verbose=True,
        allow_delegation=True,
        tools=[linkup_search_tool],
        llm=llm,
    )

### Research Analyst Agent

In [14]:
research_analyst = Agent(
        role="Research Analyst",
        goal="Analyze and synthesize raw information into structured insights, along with source links (urls) as citations.",
        backstory="An expert at analyzing information, identifying patterns, and extracting key insights. If required, can delagate the task of fact checking/verification to 'Web Searcher' only. Passes the final results to the 'Technical Writer' only.",
        verbose=True,
        allow_delegation=True,
        llm=llm,
    )

### Technical Writer Agent

In [15]:
technical_writer = Agent(
        role="Technical Writer",
        goal="Create well-structured, clear, and comprehensive responses in markdown format, with citations/source links (urls).",
        backstory="An expert at communicating complex information in an accessible way.",
        verbose=True,
        allow_delegation=False,
        llm=llm,
    )

**Now, let's define the agents' tasks.**

### Web Searcher Agent - Task

In [16]:
search_task = Task(
        description="Search for comprehensive information about the user query: {query}.",
        agent=web_searcher,
        expected_output="Detailed raw search results including sources (urls).",
        tools=[linkup_search_tool]
    )

### Research Analyst Agent - Task

In [17]:
analysis_task = Task(
        description="Analyze the raw search results, identify key information, verify facts and prepare a structured analysis.",
        agent=research_analyst,
        expected_output="A structured analysis of the information with verified facts and key insights, along with source links",
        context=[search_task]
    )

### Technical Writer Agent - Task

In [18]:
writing_task = Task(
        description="Create a comprehensive, well-organized response based on the research analysis.",
        agent=technical_writer,
        expected_output="A clear, comprehensive response that directly answers the query with proper citations/source links (urls).",
        context=[analysis_task]
    )

**Now, let's create crew to manage agents and task workflow.**

In [19]:
crew = Crew(
        agents=[web_searcher, research_analyst, technical_writer],
        tasks=[search_task, analysis_task, writing_task],
        verbose=True,
        process=Process.sequential
    )

In [20]:
# Run the crew with an example query
result = crew.kickoff(inputs={"query": "What are the best books from industry writers to learn Generative AI?"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 2059d1e5-8440-4782-a385-7a2d8b3b3351                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Generating description...
Method called by: DelegateWorkTool
{'task': {'description': 'The task to delegate', 'type': 'str', 'required': True}, 'context': {'description': 'The context for the task', 'type': 'str', 'required': True}, 'coworker': {'description': 'The role/name of the coworker to delegate to', 'type': 'str', 'required': True}}
Tool Name: Delegate work to coworker
Tool Arguments: {'task': {'description': 'The task to delegate', 'type': 'str', 'required': True}, 'context': {'description': 'The context for the task', 'type': 'str', 'required': True}, 'coworker': {'description': 'The role/name of the coworker to delegate to', 'type': 'str', 'required': True}}
Tool Description: Delegate a specific task to one of the following coworkers: Research Analyst, Technical Writer
The input to this tool should be the coworker, the task you want them to do, and ALL necessary context to execute the task, they know nothing about the task, so share absolutely everything you know, don't refe

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Web Searcher                                                                                            │
│                                                                                                                 │
│  Task: Search for comprehensive information about the user query: What are the best books from industry         │
│  writers to learn Generative AI?.                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Web Searcher                                                                                            │
│                                                                                                                 │
│  Thought: Action: Linkup Search                                                                                 │
│                                                                                                                 │
│  Using Tool: Linkup Search                                                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Technical Writer                                                                                        │
│                                                                                                                 │
│  Task: Create a comprehensive, well-organized response based on the research analysis.                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Technical Writer                                                                                        │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ## Comprehensive Analysis of Key Books on Generative AI                                                        │
│                                                                                                                 │
│  The field of Generative AI is rapidly evolving, and understanding its principles, applications, and            │
│  implications is crucial for both enthusiasts and professionals. Below is a structured analysis of key books    │
│  that offer valuable insights into Generative AI. Each entry includes essential information about the book,     │
│  key insights, and sources for further reading.                                                                 │
│                                                                                                                 │
│  ### 1. Generative AI on AWS                                                                                    │
│  - **Authors**: Chris Fregly, Antje Barth, and Shelbee Eigenbrode                                               │
│  - **Insights**: This book provides a detailed exploration of how to build and deploy generative AI models      │
│  using Amazon Web Services (AWS). It offers practical guidance on leveraging AWS’s suite of tools to create     │
│  and scale AI applications effectively.                                                                         │
│  - **Source**: [Analytics                                                                                       │
│  Vidhya](https://www.analyticsvidhya.com/blog/2023/12/books-every-genai-enthusiast-should-read/)                │
│                                                                                                                 │
│  ### 2. Ripples of Generative AI: How Generative AI Impacts, Informs and Transforms Our Lives                   │
│  - **Author**: Jacob Emerson                                                                                    │
│  - **Insights**: This book discusses the transformative impacts of generative AI across various aspects of      │
│  life, providing valuable insights into its applications and broader implications in society.                   │
│  - **Source**: [Analytics                                                                                       │
│  Vidhya](https://www.analyticsvidhya.com/blog/2023/12/books-every-genai-enthusiast-should-read/)                │
│                                                                                                                 │
│  ### 3. Artificial Intelligence Fundamentals for Business Leaders                                               │
│  - **Author**: I Almeida                                                                                        │
│  - **Insights**: Focusing on the evolution of AI, this book emphasizes data management and deep learning        │
│  principles. It is particularly useful for business leaders looking to understand the essentials of generative  │
│  AI in a business context.                                                                                      │
│  - **Source**: [Analytics                                                                                       │
│  Vidhya](https://www.analyticsvidhya.com/blog/2023/12/books-every-genai-enthusiast-should-read/)                │
│                                                        

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 6a2313a1-289f-4e13-8590-978133d03829                                                                     │
│  Agent: Technical Writer                                                                                        │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 2059d1e5-8440-4782-a385-7a2d8b3b3351                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ## Comprehensive Analysis of Key Books on Generative AI                                          │
│                                                                                                                 │
│  The field of Generative AI is rapidly evolving, and understanding its principles, applications, and            │
│  implications is crucial for both enthusiasts and professionals. Below is a structured analysis of key books    │
│  that offer valuable insights into Generative AI. Each entry includes essential information about the book,     │
│  key insights, and sources for further reading.                                                                 │
│                                                                                                                 │
│  ### 1. Generative AI on AWS                                                                                    │
│  - **Authors**: Chris Fregly, Antje Barth, and Shelbee Eigenbrode                                               │
│  - **Insights**: This book provides a detailed exploration of how to build and deploy generative AI models      │
│  using Amazon Web Services (AWS). It offers practical guidance on leveraging AWS’s suite of tools to create     │
│  and scale AI applications effectively.                                                                         │
│  - **Source**: [Analytics                                                                                       │
│  Vidhya](https://www.analyticsvidhya.com/blog/2023/12/books-every-genai-enthusiast-should-read/)                │
│                                                                                                                 │
│  ### 2. Ripples of Generative AI: How Generative AI Impacts, Informs and Transforms Our Lives                   │
│  - **Author**: Jacob Emerson                                                                                    │
│  - **Insights**: This book discusses the transformative impacts of generative AI across various aspects of      │
│  life, providing valuable insights into its applications and broader implications in society.                   │
│  - **Source**: [Analytics                                                                                       │
│  Vidhya](https://www.analyticsvidhya.com/blog/2023/12/books-every-genai-enthusiast-should-read/)                │
│                                                                                                                 │
│  ### 3. Artificial Intelligence Fundamentals for Business Leaders                                               │
│  - **Author**: I Almeida                                                                                        │
│  - **Insights**: Focusing on the evolution of AI, this book emphasizes data management and deep learning        │
│  principles. It is particularly useful for business leaders looking to understand the essentials of generative  │
│  AI in a business context.                                                                                      │
│  - **Source**: [Analytics                                                                                       │
│  Vidhya](https://www.analyticsvidhya.com/blog/2023/12/

In [26]:
#Print result
display(Markdown(result.raw))

## Comprehensive Analysis of Key Books on Generative AI

The field of Generative AI is rapidly evolving, and understanding its principles, applications, and implications is crucial for both enthusiasts and professionals. Below is a structured analysis of key books that offer valuable insights into Generative AI. Each entry includes essential information about the book, key insights, and sources for further reading.

### 1. Generative AI on AWS
- **Authors**: Chris Fregly, Antje Barth, and Shelbee Eigenbrode  
- **Insights**: This book provides a detailed exploration of how to build and deploy generative AI models using Amazon Web Services (AWS). It offers practical guidance on leveraging AWS’s suite of tools to create and scale AI applications effectively.  
- **Source**: [Analytics Vidhya](https://www.analyticsvidhya.com/blog/2023/12/books-every-genai-enthusiast-should-read/)

### 2. Ripples of Generative AI: How Generative AI Impacts, Informs and Transforms Our Lives
- **Author**: Jacob Emerson  
- **Insights**: This book discusses the transformative impacts of generative AI across various aspects of life, providing valuable insights into its applications and broader implications in society.  
- **Source**: [Analytics Vidhya](https://www.analyticsvidhya.com/blog/2023/12/books-every-genai-enthusiast-should-read/)

### 3. Artificial Intelligence Fundamentals for Business Leaders
- **Author**: I Almeida  
- **Insights**: Focusing on the evolution of AI, this book emphasizes data management and deep learning principles. It is particularly useful for business leaders looking to understand the essentials of generative AI in a business context.  
- **Source**: [Analytics Vidhya](https://www.analyticsvidhya.com/blog/2023/12/books-every-genai-enthusiast-should-read/)

### 4. Introduction to Generative AI
- **Authors**: Numa Dhamani and Maggie Engler  
- **Insights**: This book serves as an accessible introduction to generative AI, providing a focus on societal impacts and associated risks. It is intended for a broad audience, making complex topics in AI more approachable.  
- **Source**: [Manning Publications](https://www.manning.com/books/introduction-to-generative-ai)

### 5. Generative AI: The Insights You Need from Harvard Business Review
- **Source**: Harvard Business Review  
- **Insights**: A compilation of key insights and case studies that illustrate how businesses are transformed by generative AI. This book serves as a practical guide for managers seeking to implement AI strategies effectively.  
- **Source**: [Harvard Business Review](https://store.hbr.org/product/generative-ai-the-insights-you-need-from-harvard-business-review/10697)

### 6. Generative AI in Practice
- **Author**: David A. Teich  
- **Insights**: The book provides a comprehensive overview of the implications of generative AI across multiple sectors, offering insights valuable for industry professionals looking to leverage AI’s transformative potential.  
- **Source**: [Forbes](https://www.forbes.com/sites/davidteich/2024/05/22/generative-ai-in-practice-a-good-book-just-ignore-the-buzzword/)

### 7. Generative AI with LangChain
- **Source**: Undisclosed Authors  
- **Insights**: This book focuses on utilizing LangChain for generative AI applications, providing practical lessons and best practices that developers can adopt for successful implementation.  
- **Source**: [Medium](https://ivopbernardo.medium.com/the-books-you-need-to-read-to-master-generative-ai-a6dbf437c8ea)  

### Conclusion
These titles collectively provide a comprehensive view of Generative AI, suited for varied audiences from business leaders to developers. By exploring these resources, readers can deepen their understanding of generative AI technology and its transformative effects across different domains. For ongoing developments in this field, keeping abreast with current literature and practices is recommended.

--- 

This response aims to deliver a thorough understanding of the selected literature on Generative AI, backed with citations for further exploration.